In [ ]:
# Necessary imports
import sys
sys.path.append('src/')
import utils, models, layers, metrics
import os
import pickle
import numpy as np

In [ ]:
# Hyperparameters:
# Training
batch_size = 16
epochs = 500
lr_g = 0.0001
lr_d = 0.0004

# Loss-Function
lambda_gp = 10
n_disc = 1
noise_stddev = 0.0001

# Output and build related
z_dim = 100
h_dim = 128
w_dim = 128
c_dim = 3
scale = 4 # image gets downsampled 16 (2^4) times before prediction, dense embedding is then 4*4*channels
channels = 1024
n_samples = 20

# CGAN
n_classes = 120
embedding_dim = 50

# SAGAN
c_scale = 8

In [ ]:
# build the model
model = models.WGANGP(z_dim = z_dim, h_dim=h_dim, w_dim=w_dim, c_dim=c_dim, channels=channels, scale=scale, 
                         epochs=epochs, batch_size=batch_size, lr_g=lr_g, lr_d=lr_d,
                         lambda_gp=lambda_gp, noise_stddev=noise_stddev, n_disc=n_disc, n_samples=n_samples)

In [ ]:
model = models.CWGANGP(z_dim = z_dim, h_dim=h_dim, w_dim=w_dim, c_dim=c_dim, channels=channels, scale=scale, 
                         n_classes = n_classes, embedding_dim = 50, epochs=epochs, batch_size=batch_size, lr_g=lr_g, lr_d=lr_d,
                         lambda_gp=lambda_gp, noise_stddev=noise_stddev, n_disc=n_disc, n_samples=n_samples)

In [ ]:
model = models.CSAWGANGP(z_dim = z_dim, h_dim=h_dim, w_dim=w_dim, c_dim=c_dim, channels=channels, scale=scale, 
                         n_classes = n_classes, embedding_dim = 50, c_scale=c_scale, epochs=epochs, batch_size=batch_size, lr_g=lr_g, lr_d=lr_d,
                         lambda_gp=lambda_gp, noise_stddev=noise_stddev, n_disc=n_disc, n_samples=n_samples)

In [ ]:
model = models.ProjSNGAN(z_dim = z_dim, h_dim=h_dim, w_dim=w_dim, c_dim=c_dim, channels=channels, scale=scale, 
                         n_classes = n_classes, embedding_dim = 50, c_scale=c_scale, epochs=epochs, batch_size=batch_size, lr_g=lr_g, lr_d=lr_d,
                         lambda_gp=lambda_gp, noise_stddev=noise_stddev, n_disc=n_disc, n_samples=n_samples)

In [ ]:
!pip install pydot
!pip install pydotplus
!pip install graphviz

In [ ]:
# read images
reader = utils.DataReader(batch_size, "TFRecords/ImageNet64_Train")
model.train(reader.data, log_n = 50, verbose=50) 
    

In [ ]:
import tensorflow as tf
tf.keras.utils.plot_model(model.d, show_shapes=True, dpi=64)

In [ ]:
tf.keras.utils.plot_model(model.g, show_shapes=True, dpi=64)

In [ ]:
!pip install tensorflow-datasets

In [ ]:
!pip install tensorflow-addons

In [ ]:
import tensorflow_datasets as tfds

ds_train, ds_info = tfds.load('mnist', split='train', shuffle_files=True, as_supervised=True, with_info=True)


In [ ]:
import tensorflow_datasets as tfds

ds_train, ds_info = tfds.load('stanford_dogs', split='train+test', shuffle_files=True, as_supervised=True, with_info=True)

In [ ]:
import tensorflow_datasets as tfds

ds_train, ds_info = tfds.load('fashion_mnist', split='train+test', shuffle_files=True, as_supervised=True, with_info=True)

In [ ]:
import tensorflow_datasets as tfds

ds_train, ds_info = tfds.load('cifar10', split='train+test', shuffle_files=True, as_supervised=True, with_info=True)

In [ ]:
import tensorflow_datasets as tfds

ds_train, ds_info = tfds.load('stl10', split='train+test', shuffle_files=True, as_supervised=True, with_info=True)

In [ ]:

import tensorflow_datasets as tfds

ds_train, ds_info = tfds.load('celeb_a', split='train', shuffle_files=True)


In [ ]:
import tensorflow as tf
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    #image = tf.image.resize(image, [w_dim, h_dim])
    image = (tf.cast(image, tf.float32) / 128.) - 1.
    #image_list = []
    #image_list.append(image)
    #for i in range(1,scale+1):
        #image = tf.image.resize(image, [w_dim//pow(2,i), h_dim//pow(2,i)])
        #image_list.append(image)
    image = tf.image.resize(image, [w_dim, h_dim])
    #return (tf.cast(image, tf.float32) / 128.) - 1., label
    #return image_list[0],image_list[1], image_list[2],image_list[3],image_list[4],label
    return image, label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

model.train(ds_train, verbose= 50 ,log_n = 50) 